In [1]:
#import packages
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Point
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime

import warnings
warnings.filterwarnings('ignore')

### NYC Open Data:
    
Subway and Bus Stops SODA API Urls:

* 311 Call Inquiries: `https://data.cityofnewyork.us/resource/wewp-mm3p.json`

* 311 Web Inquiries: `https://data.cityofnewyork.us/resource/vwpc-kje2.json`

* All 311 Service Requests: `https://data.cityofnewyork.us/resource/erm2-nwe9.json`


Supporting Shapefiles:

* NYC Census Tracts 2020: `https://data.cityofnewyork.us/resource/63ge-mke6.json`

In [18]:
def get_data(url, uniqueid, startdatetime, enddatetime, cat_clnm, subcat_clnm, subsubcat_clnm, datetime_clnm):
    '''
    Inputs: 
    url - string value of SODA API url
    startdatetime & enddatetime - string values of the format 'YYYY-MM-DDTHH:MM:SS', i.e. '2019-12-31T23:59:59.000' 
                                for the beginning and end of timeframe of interest
    datetime_clnm - string value of the column name in the data for the datetime of complaint inquiry
    
    Output:
    gdf - GeoDataFrame of 311 inquiries. Geometry is point location of 311 inquiry.
    '''
    query = (url +'?'
            "$select= %s as ID,%s as cat, %s as subcat, %s as subsubcat, %s as datetime"
            "&$where=datetime between '"+startdatetime+"' and '"+enddatetime+"'"
            "&$order=datetime"
            "&$limit=50000"
            )%(uniqueid,cat_clnm,subcat_clnm,subsubcat_clnm,datetime_clnm)
    query = query.replace(" ", "%20")
    print(query)
    response = urllib.request.urlopen(query)
    data = json.loads(response.read())

#     for d in data:
#         d['geometry'] = shape(d['the_geom'])

#     #store in a geodataframe
#     gdf = gpd.GeoDataFrame(data, geometry = 'geometry', crs = 'EPSG: 4326')
#     gdf = gdf.drop(columns = ['the_geom'])
    
    return data

In [19]:
calls =  'https://data.cityofnewyork.us/resource/wewp-mm3p.json'
web = 'https://data.cityofnewyork.us/resource/vwpc-kje2.json'

uniqueid = 'UNIQUE_ID'
cat_clnm = 'AGENCY_NAME'
subcat_clnm = 'INQUIRY_NAME'
subsubcat_clnm = 'BRIEF_DESCRIPTION'
datetime_clnm = 'DATE_TIME'

data = get_data(calls, uniqueid, '2019-01-01T00:00:00.000', '2019-02-01T00:00:00.000', cat_clnm, subcat_clnm, subsubcat_clnm, datetime_clnm)
# query = (calls +'?'
#             "$select=count(*) as count,%s as cat, %s as subcat, %s as subsubcat"
#             "&$group=cat,subcat,subsubcat"
#             "&$limit=500"
#             )%(cat_clnm,subcat_clnm,subcat_clnm)
# query = query.replace(" ", "%20")
# response = urllib.request.urlopen(query)
# data = json.loads(response.read())


https://data.cityofnewyork.us/resource/wewp-mm3p.json?$select=%20UNIQUE_ID%20as%20ID,AGENCY_NAME%20as%20cat,%20INQUIRY_NAME%20as%20subcat,%20BRIEF_DESCRIPTION%20as%20subsubcat,%20DATE_TIME%20as%20datetime&$where=datetime%20between%20'2019-01-01T00:00:00.000'%20and%20'2019-02-01T00:00:00.000'&$order=datetime&$limit=50000


In [20]:
len(data)

50000

In [21]:
call_df = pd.DataFrame(data, columns = data[0].keys())
call_df.head()

,ID,cat,subcat,subsubcat,datetime
0,134668788,New York City Police Department,Division Transfer,Hidden service for activity coding.,2019-01-01T00:00:01.000
1,134670503,New York City Police Department,Vehicle Blocking Driveway Complaint,Report a vehicle that is blocking a driveway.,2019-01-01T00:00:32.000
2,134670111,Department of Environmental Protection,Sewer Backup Complaint - Priority,Report a sewer backup within a building.,2019-01-01T00:00:57.000
3,134669317,New York City Police Department,Suicide,Caller or acquaintance is considering suicide ...,2019-01-01T00:01:02.000
4,134669322,Human Resources Administration,Adult Protective Services,Assistance for adults 18 and older with mental...,2019-01-01T00:01:08.000


In [25]:
call_IDs = call_df['ID'].unique().tolist()
print(len(call_IDs))
print(call_IDs[0:10])

50000
['134668788', '134670503', '134670111', '134669317', '134669322', '134669235', '134669074', '134669075', '134670325', '134669236']


In [46]:
all311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
startdatetime = '2019-01-01T00:00:00.000'
enddatetime = '2019-02-01T00:00:00.000'
query = (all311 +'?'
            "$select=*"
            "&$where=created_date between '"+startdatetime+"' and '"+enddatetime+"' AND open_data_channel_type = 'PHONE'"            
            "&$order=created_date"
            "&$limit=5000000"
            )
query = query.replace(" ", "%20")
print(query)
response = urllib.request.urlopen(query)
data = json.loads(response.read())

https://data.cityofnewyork.us/resource/erm2-nwe9.json?$select=*&$where=created_date%20between%20'2019-01-01T00:00:00.000'%20and%20'2019-02-01T00:00:00.000'%20AND%20open_data_channel_type%20=%20'PHONE'&$order=created_date&$limit=5000000


In [66]:
all_df = pd.DataFrame(data, columns = data[0].keys())
all_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location
0,41314898,2019-01-01T00:00:00.000,2019-01-01T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,Other (Explain Below),11208,790 ELDERT LANE,...,3042719001,BROOKLYN,1022225,183607,PHONE,Unspecified,BROOKLYN,40.67055409879978,-73.8631053928292,"{'latitude': '40.67055409879978', 'longitude':..."
1,41315045,2019-01-01T00:00:00.000,2019-01-03T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,Commercial Building,11412,NaN,...,NaN,QUEENS,1046613,196294,PHONE,Unspecified,QUEENS,40.70523868495376,-73.77507291150168,"{'latitude': '40.70523868495376', 'longitude':..."
2,41315216,2019-01-01T00:00:00.000,2019-01-16T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Unsanitary Pigeon Condition,Pigeon Waste,3+ Family Apartment Building,11694,152 BEACH 118 STREET,...,4162270058,QUEENS,1029147,150009,PHONE,Unspecified,QUEENS,40.5783023947916,-73.83837606412298,"{'latitude': '40.5783023947916', 'longitude': ..."
3,41315249,2019-01-01T00:00:00.000,2019-01-03T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Other (Explain Below),11205,321 CARLTON AVENUE,...,3021030003,BROOKLYN,992058,190002,PHONE,Unspecified,BROOKLYN,40.68818492678691,-73.97184581178426,"{'latitude': '40.68818492678691', 'longitude':..."
4,41315356,2019-01-01T00:00:00.000,2019-01-03T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Unsanitary Pigeon Condition,Pigeon Waste,Commercial Building,11205,975 BEDFORD AVENUE,...,3017780009,BROOKLYN,996630,190915,PHONE,Unspecified,BROOKLYN,40.690685687266026,-73.9553583591566,"{'latitude': '40.690685687266026', 'longitude'..."


In [67]:
print(len(all_df))

133847


In [68]:
overlapping_data = pd.merge(all_df,call_df, left_on = ['agency_name','complaint_type'], right_on = ['cat','subcat'], how = 'inner')
print(len(overlapping_data))
overlapping_data.head()

40176


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,park_facility_name,park_borough,latitude,longitude,location,ID,cat,subcat,subsubcat,datetime
0,41308504,2019-01-01T00:34:26.000,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134669474,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01T00:33:02.000
1,41308504,2019-01-01T00:34:26.000,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134679166,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01T10:14:57.000
2,41308504,2019-01-01T00:34:26.000,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134671441,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01T10:38:27.000
3,41308504,2019-01-01T00:34:26.000,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134672210,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01T10:56:14.000
4,41308504,2019-01-01T00:34:26.000,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134674591,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01T13:31:36.000


In [69]:
overlapping_data.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'address_type', 'city', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'latitude', 'longitude',
       'location', 'ID', 'cat', 'subcat', 'subsubcat', 'datetime'],
      dtype='object')

In [74]:
#convert to datetime
overlapping_data['datetime'] = pd.to_datetime(overlapping_data['datetime'])
overlapping_data['created_date'] = pd.to_datetime(overlapping_data['created_date'])

#limit dataframe to call date and input date being the same and time difference should be less than or equal to 10 mins
overlapping_data = overlapping_data[(overlapping_data['datetime'].dt.date==overlapping_data['created_date'].dt.date)&
                                   ((overlapping_data['created_date']-overlapping_data['datetime']).dt.total_seconds()/60<=10)&
                                    ((overlapping_data['created_date']-overlapping_data['datetime']).dt.total_seconds()/60>0)]

print(len(overlapping_data))

89


In [75]:
overlapping_data.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,park_facility_name,park_borough,latitude,longitude,location,ID,cat,subcat,subsubcat,datetime
0,41308504,2019-01-01 00:34:26,2019-01-01T08:22:02.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11234,2213 EAST 69 STREET,...,Unspecified,BROOKLYN,40.61686849087115,-73.90666237838713,"{'latitude': '40.61686849087115', 'longitude':...",134669474,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01 00:33:02
49,41316205,2019-01-01 10:16:56,2019-01-01T11:50:19.000,NYPD,New York City Police Department,Animal Abuse,Tortured,Residential Building/House,10037,2101 MADISON AVENUE,...,Unspecified,MANHATTAN,40.81021106498936,-73.93736778909592,"{'latitude': '40.81021106498936', 'longitude':...",134679166,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01 10:14:57
98,41318725,2019-01-01 10:39:32,2019-01-01T12:58:04.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11423,NaN,...,Unspecified,QUEENS,40.72433863570052,-73.76907003919959,"{'latitude': '40.72433863570052', 'longitude':...",134671441,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01 10:38:27
148,41318388,2019-01-01 13:38:24,2019-01-01T14:25:57.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11429,216-31 111 AVENUE,...,Unspecified,QUEENS,40.70705801674601,-73.74091353227682,"{'latitude': '40.70705801674601', 'longitude':...",134674591,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01 13:31:36
149,41318388,2019-01-01 13:38:24,2019-01-01T14:25:57.000,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11429,216-31 111 AVENUE,...,Unspecified,QUEENS,40.70705801674601,-73.74091353227682,"{'latitude': '40.70705801674601', 'longitude':...",134678084,New York City Police Department,Animal Abuse,Report an abused animal.,2019-01-01 13:37:53
